# Seemingly Unrelated Regression

In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.system import SUR

df = pd.read_csv('sur_data.csv')
df.head()

,milk_q,milk_p,income,beer_q,beer_p
0,120,5,45,80,8
1,150,6,50,95,9
2,100,5,48,85,8
3,130,7,52,90,10
4,160,6,47,100,9


In [31]:
df = sm.add_constant(df)

#Define the system of equations
formulas = {
    'eq1': 'milk_q ~ milk_p + income + const',
    'eq2': 'beer_q ~ beer_p + income + const'
}


#Fit the SUR model
model = SUR.from_formula(formulas, data=df)
results = model.fit()
results.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                           System GLS Estimation Summary                           
===================================================================================
Estimator:                        GLS   Overall R-squared:                   0.5364
No. Equations.:                     2   McElroy's R-squared:                 0.4160
No. Observations:                  10   Judge's (OLS) R-squared:             0.5364
Date:                Wed, May 29 2024   Berndt's R-squared:                  0.5894
Time:                        16:40:26   Dhrymes's R-squared:                 0.5364
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
                  Equation: eq1, Dependent Variable: milk_q                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
milk_p         21.512     5.5292     3.8906     0.0001      10.675      32.349
income        -2.7683     2.1214    -1.3049     0.1919     -6.9262      1.3896
const          142.27     92.676     1.5351     0.1248     -39.373      323.91
                  Equation: eq2, Dependent Variable: beer_q                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
beer_p         8.4268     2.4187     3.4841     0.0005      3.6863      13.167
income        -0.8902     1.0388    -0.8570     0.3914     -2.9262      1.1457
const          56.110     36.347     1.5437     0.1227     -15.129      127.35
==============================================================================

Covariance Estimator:
Heteroskedastic (Robust) Covariance (Debiased: False, GLS: True)
"""

In [33]:
y = df['milk_q']
X = df[['milk_p','income']]
X = sm.add_constant(X)

sm.OLS(y,X).fit().summary()

/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 milk_q   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     4.076
Date:                Wed, 29 May 2024   Prob (F-statistic):             0.0670
Time:                        16:42:01   Log-Likelihood:                -39.491
No. Observations:                  10   AIC:                             84.98
Df Residuals:                       7   BIC:                             85.89
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        142.2683    114.520      1.242      0.254    -128.529     413.065
milk_p        21.5122      7.808      2.755      0.028       3.048      39.976
income        -2.7683      2.773     -0.998      0.351      -9.326       3.789
==============================================================================
Omnibus:                        0.995   Durbin-Watson:                   2.687
Prob(Omnibus):                  0.608   Jarque-Bera (JB):                0.629
Skew:                           0.025   Prob(JB):                        0.730
Kurtosis:                       1.772   Cond. No.                     1.18e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""